#### Fixing the recovery graphing portion of the package
- Dates for recovery should lag from cycles (at least we think at this point)

In [1]:
import numpy as np 
import pandas as pd
from IPython.display import display, HTML

import plotly.graph_objs as go
import plotly.express as px


In [2]:
# Reading in our df 
df = pd.read_pickle("../data/clean/records/recovery/2021-12-27_2022-08-01.csv")

In [3]:
# Filter dates 

# All data after select date
# df = df.loc[df['during_start'] > '2022-05-01'] # cjx data 

# Between two dates 
df = df.loc[df["during_start"].between("2022-05-06", "2022-05-31")]

In [4]:
# Adding in helper columns for day_start
df['ds_date'] = pd.to_datetime(df['during_start'], unit='ms').dt.date 
df['ds_frmt_time'] = pd.to_datetime(df["during_start"], unit='ms').dt.strftime('%I:%M:%S %p')
df['ds_ampm'] = pd.to_datetime(df["during_start"], unit='ms').dt.strftime('%I %p')

df['de_date'] = pd.to_datetime(df['during_end'], unit='ms').dt.date 
df['de_frmt_time'] = pd.to_datetime(df["during_end"], unit='ms').dt.strftime('%I:%M:%S %p')
df['de_ampm'] = pd.to_datetime(df["during_end"], unit='ms').dt.strftime('%I %p')

In [5]:
# View the entire df head for ref 
display(HTML(df.head().to_html()))

,id,created_at,updated_at,date,user_id,sleep_id,cycle_id,responded,score,resting_heart_rate,hrv_rmssd,state,calibrating,prob_covid,hr_baseline,skin_temp_celsius,spo2,algo_version,rhr_component,hrv_component,history_size,from_sws,rate,during_start,during_end,skin_temp_fahrenheit,ds_date,ds_frmt_time,ds_ampm,de_date,de_frmt_time,de_ampm
315,229561014,2022-05-07 07:36:56.381,2022-05-07 07:37:05.931,2022-05-07 07:28:18.117,3802464,430711434,224345865,False,76,45,0.089019,complete,False,0.0,46.0,34.300,95.909090,5.0.0,0,0,4.0,False,7.242785,2022-05-06 23:37:21.419,2022-05-07 07:28:18.117,93,2022-05-06,11:37:21 PM,11 PM,2022-05-07,07:28:18 AM,07 AM
316,230013545,2022-05-08 09:30:21.534,2022-05-08 09:30:23.527,2022-05-08 09:16:20.276,3802464,431381217,224788831,False,88,41,0.103527,complete,False,0.0,47.5,34.100,95.958336,5.0.0,0,0,5.0,False,1.258895,2022-05-08 00:25:54.224,2022-05-08 09:16:20.276,93,2022-05-08,12:25:54 AM,12 AM,2022-05-08,09:16:20 AM,09 AM
317,230401894,2022-05-09 08:32:22.008,2022-05-09 08:32:23.651,2022-05-09 08:23:05.600,3802464,431948916,225168218,False,64,45,0.084529,complete,False,0.0,49.0,34.500,96.346150,5.0.0,0,0,6.0,False,8.151113,2022-05-08 22:47:11.594,2022-05-09 08:23:05.600,94,2022-05-08,10:47:11 PM,10 PM,2022-05-09,08:23:05 AM,08 AM
318,230808053,2022-05-10 09:04:52.251,2022-05-10 09:04:53.700,2022-05-10 08:53:49.050,3802464,432623840,225582260,False,47,47,0.075018,complete,False,0.0,47.0,34.595,97.333336,5.0.0,0,0,7.0,False,5.848230,2022-05-10 02:12:49.511,2022-05-10 08:53:49.050,94,2022-05-10,02:12:49 AM,02 AM,2022-05-10,08:53:49 AM,08 AM
319,231180425,2022-05-11 08:52:29.533,2022-05-11 08:52:31.894,2022-05-11 08:41:11.438,3802464,433256233,225961980,False,48,49,0.078986,complete,False,0.0,45.0,34.615,95.736840,5.0.0,0,0,8.0,False,4.950978,2022-05-11 00:20:57.556,2022-05-11 08:41:11.438,94,2022-05-11,12:20:57 AM,12 AM,2022-05-11,08:41:11 AM,08 AM


### Sleep vs Wake Time
> sleep_wake() in VisbaleWhoop

In [6]:
#df = df.assign(ds_start = df["ds_start"] * -1) # Convert kilojules to cals

fig = go.Figure()

fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["ds_ampm"],
mode="markers+lines",
name = "Sleep Start",
text = df['ds_date'].astype('string') + " - " + df['ds_frmt_time'].astype('string'),
hoverinfo = 'text'))

fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["de_ampm"],
mode="markers+lines",
name = "Wake Time",
text = df['de_date'].astype('string') + " - " + df['de_frmt_time'].astype('string'),
hoverinfo = 'text'))

fig.update_yaxes(categoryorder='array', categoryarray= ['05 PM', '06 PM', '07PM','08 PM','09 PM', '10 PM', '11 PM', '12 AM', '01 AM', '02 AM', '03 AM', '04 AM',
                                                        '05 AM', '06 AM', '07 AM', '08 AM', '09 AM', '10 AM', '11 AM', '12 PM',
                                                        '01 PM', '02 PM', '03 PM', '04 PM'])

fig.update_layout(title_text='Sleep and Wake Times', title_x=0.5)
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Time')

fig.show()

### Recovery Score 
- mimics view in app 
> recovery_score() in VisbaleWhoop

In [7]:
# Mapping categories to strain ranges
bins = [0, 33, 66, np.inf]
names = ['red', 'yellow', 'lightgreen']

df['RecoveryCategory'] = pd.cut(df['score'], bins, labels=names)


fig = px.bar(df, x='de_date', y='score',
            labels={'de_date': 'Date', 'score':'Recovery Score'})

fig.update_traces(text = df.score, textposition = "outside", marker_color=df["RecoveryCategory"])
fig.update_layout(title_text='Recovery Score by Day', title_x=0.5)
fig.show()

### Covid Probability by Percentage
> covid_prob in VisableWhoop

In [8]:
df['prob_covid'] = df['prob_covid'].fillna(0)

fig = px.line(df, x='de_date', y="prob_covid",
            labels={'de_date': 'Date', 'prob_covid':'Covid Probability'})


fig.layout.yaxis.tickformat = ',.0%'
fig.update_layout(title_text='Covid Probability', title_x=0.5)

fig.show()

### HRV RMMSD
- the root mean square of successive differences between heartbeats
> hrv() in VisableWhoop

In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["hrv_rmssd"],
mode="markers+lines"))

fig.update_layout(title_text='HRV (RMSSD)', title_x=0.5)
fig.update_xaxes(title='Date')
fig.update_yaxes(title='RMSSD')

### Resting HR vs Baseline HR
> resting_base_HR() in VisableWhoop

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["resting_heart_rate"],
mode="markers+lines",
name = "Resting HR",
text = df['resting_heart_rate'].astype('string'),
hoverinfo = 'text'))

fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["hr_baseline"],
mode="markers+lines",
name = "Baseline HR",
text = df['hr_baseline'].astype('string'),
hoverinfo = 'text'))

fig.update_layout(title_text='Resting vs Baseline Heart Rate (HR)', title_x=0.5)
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Heart Rate')

### Skin Temp in Fahrenheit and SPO2
- celcius is also avaliable in the data 
> skintemp_spo() in VisableWhoop
#### NOTE: Whoop 4.0 ONLY

In [11]:
# resting_heart_rate, hr_baseline, skin_temp_celsius, spo2
# 1.8 + 32
# df = df.assign(skin_temp_fahrenheit = (df["skin_temp_celsius"]*1.8) + 32) # Convert kilojules to cals

fig = go.Figure()


fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["skin_temp_fahrenheit"],
mode="markers+lines",
name = "Skin Temp (Fahrenheit)",
text = df['skin_temp_fahrenheit'].astype('string'),
hoverinfo = 'text'))

fig.add_trace(go.Scatter(
x=df.ds_date,
y=df["spo2"],
mode="markers+lines",
name = "SPO2",
text = df['spo2'].astype('string'),
hoverinfo = 'text'))

fig.update_layout(title_text='Skin Temp (F) & SPO2', title_x=0.5)

fig.show()



##### In order to get the next two graphs we need both cycles_df and recovery_df 
- set prefixs in order to distinguish between two dfs once merced  

In [12]:
cycles_df = pd.read_pickle("../data/clean/records/cycles/2021-12-27_2022-08-01.csv")
recovery_df = pd.read_pickle("../data/clean/records/recovery/2021-12-27_2022-08-01.csv")

cycles_df = cycles_df.add_prefix("c_")
recovery_df = recovery_df.add_prefix("r_")

df = pd.merge(cycles_df, recovery_df, how="left", left_on="c_cycle_id", right_on="r_cycle_id")

In [13]:
# Again can filter data if we want
# Filter dates 

# All data after select date
# df = df.loc[df['r_during_start'] > '2022-05-01'] # cjx data 

# Between two dates 
df = df.loc[df["r_during_start"].between("2022-05-06", "2022-05-30")]

### Heart Rate Stats by Day
- Daily Average Heart Rate
- Daily Max Heart Rate
- Daily Resting HR
> hr_breakdown() in VisableWhoop

In [14]:
fig = px.bar(df, x='c_days_start', y=['c_day_avg_heart_rate', 'c_day_max_heart_rate', 'r_resting_heart_rate'], orientation = "v",
    barmode = 'group',)

newnames = {'c_day_avg_heart_rate':'Day Avg HR', 'c_day_max_heart_rate': 'Day MAX HR', 'r_resting_heart_rate':'Resting HR'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))
fig.update_layout(title_text='Heart Rate Stats by Day', title_x=0.5, xaxis_title="", yaxis_title="Heart Rate", legend_title="HR Type")
fig.show()

### Scaled Strain and Recovery Score 
- again we need both cycles_df and recovery_df therefore we use the same df as above
> scaledstrain_recovery() in VisableWhoop

In [15]:
# cycles day_strain
# recovery scpre

fig = go.Figure()

fig.add_trace(go.Scatter(
x=df.c_days_start,
y=df["c_scaled_strain"],
mode="markers+lines",
name = "Scaled Straint",
text = df['c_scaled_strain'].round(2),
hoverinfo = 'text',
yaxis="y1"))


fig.add_trace(go.Scatter(
x=df.c_days_start,
y=df["r_score"],
mode="markers+lines",
name = "Recovery Score",
text = df['r_score'].astype(int),
hoverinfo = 'text',
yaxis="y2"))

# Create axis objects
fig.update_layout(
    xaxis=dict(
    domain=[.2, 1]
    ),
    yaxis=dict(                         
        title="Scaled Strain", 
        titlefont=dict(
            color="#d99b16" 
        ),
        tickfont=dict(
            color="#d99b16"
        )
    ),
    yaxis2=dict(
        title="Recovery Score", 
        titlefont=dict(
            color="#f70f13"  
        ),
        tickfont=dict(
            color="#f70f13"
        ),
        anchor="free", 
        overlaying="y",
        side="left",
        position=0.15
    ),
    title_text='Scaled Strain & Recovery Score', title_x=0.5,
    )